In [1]:
import numpy as np
from matplotlib import pyplot as plt

import import_ipynb
import Network_Jammed as Network

importing Jupyter notebook from Network_Jammed.ipynb


In [2]:
# oscillating adaptive training. Takes an input network, cycle train length, number of oscillations and applied training strain (eps)
# returns networks through training, bulk moduli through training, and shear moduli through training
def osc(net, train_length, num_osc, eps):
    T = train_length * num_osc
    train_type = True
    working_net = net.copy()
    nets = [working_net.copy()]
    Bs = []
    Gs = []
    
    for t in range(T):
        # oscillates training direction
        if t%train_length == 0:
            train_type = not(train_type)
            
        # trains for a low poisson ratio
        if train_type == True:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(bulk_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
            
        # trains for a high poisson ratio
        if train_type == False:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(shear_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
            
    bulk_net, B = nets[-1].bulk_modulus(eps)
    shear_net, G = nets[-1].shear_modulus(eps)
    Bs.append(B)
    Gs.append(G)
    
    return np.array(nets), np.array(Bs), np.array(Gs) 

In [3]:
# control training. Takes a network, cycle training time, an applied training strain (eps), and a training direction 
#(-1 is for a low poisson ratio, +1 is a high poisson ratio)
def train(net, T, eps, train_type = -1):
    working_net = net.copy()
    nets = [working_net.copy()]
    Bs = []
    Gs = []
    
    for t in range(T):
        
        # trains for a low poisson ratio id train_type = -1
        if train_type == -1:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(bulk_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
            
        # trains for a low poisson ratio id train_type = +1
        if train_type == 1:
            bulk_net, B = nets[-1].bulk_modulus(eps)
            shear_net, G = nets[-1].shear_modulus(eps)
            Bs.append(B)
            Gs.append(G)
            cut_bond_ind = np.argmax(np.abs(shear_net.stresses()))
            nets.append(nets[-1].prune(cut_bond_ind))
            
    bulk_net, B = nets[-1].bulk_modulus(eps)
    shear_net, G = nets[-1].shear_modulus(eps)
    Bs.append(B)
    Gs.append(G)
    
    return np.array(nets), np.array(Bs), np.array(Gs) 

In [4]:
# calculates poisson ratios
def poisson_ratios(Gs, Bs):
    v = (1 - Gs/Bs) / (1 + Gs/Bs)
    return v

In [5]:
net = Network.get_network(200, '1', 5)
trained_nets, Bs, Gs = osc(net, 20, 4, -1e-6)

In [6]:
vs = poisson_ratios(Gs, Bs)